In [1]:
import json

import ndjson


def read_ndjson(path):
    # read  ndjson file into an dict
    with open(path, 'r') as f:
        data = ndjson.load(f)
        print(path)
        return data[0]

import os



labelbox_dir = 'labelbox/'
labelbox_dicts= []
labelme_dicts= []
for filename in os.listdir(labelbox_dir):
    if filename.endswith('.ndjson'):
        file_path = os.path.join(labelbox_dir, filename)
        data= read_ndjson(file_path)
        labelbox_dicts.append(data)
        #print(data)
for filename in os.listdir("labelme"):
    if filename.endswith('.json'):
        file_path = os.path.join("labelme", filename)
        print(file_path)
        with open(file_path, 'r') as f:
            data= json.load(f)
            labelme_dicts.append(data)
            #print(data)

    

labelbox/export-result-2.ndjson
labelbox/export-result.ndjson
labelme/swissimage-dop10_2022_2679-1253_0.1_2056.json
labelme/Swiss_image_6.json
labelme/Swiss_image_7.json
labelme/image_1_label.json
labelme/swissimage-dop10_2022_2675-1246_0.1_2056.json
labelme/swissimage-dop10_2022_2674-1247_0.1_2056.json
labelme/image_6_label.json
labelme/image_0_labels.json
labelme/image_3_label.json
labelme/Swiss_image_1.json
labelme/Swiss_image_2.json
labelme/swissimage-dop10_2021_2671-1248_0.1_2056.json
labelme/image_5_label.json
labelme/Swiss_image_3.json
labelme/Swiss_image_8.json
labelme/swissimage-dop10_2022_2674-1251_0.1_2056.json
labelme/Swiss_image_4.json
labelme/Swiss_image_5.json


In [2]:
labelme_dicts[-2]

In [3]:
labelme_polygons = [x["shapes"] for x in labelme_dicts]

In [4]:
len(labelme_polygons)

In [5]:
labelbox_dicts[1]

In [6]:
# transform labelbox dicts into the format of labelme polygons
ids = [list(x["projects"].keys())[0] for x in labelbox_dicts]
ids

In [ ]:
labelbox_polygons = [labelbox_dicts[x]["projects"][ids[x]]["labels"][0]['annotations']["objects"] for x in range(len(labelbox_dicts))]
# drop some infos 
labelbox_polygons = [[x["polygon"] for x in labelbox_polygons[y]] for y in range(len(labelbox_polygons))]

# add label to each entry
labelbox_polygons = [[{"label": "car", "points": x} for x in labelbox_polygons[y]] for y in range(len(labelbox_polygons))]
labelbox_polygons

In [ ]:
# remove x and y from points
labelbox_polygons_formatted= []
for image in labelbox_polygons:
    image_list = []
    for polygon in image:
        polygon_list= []
        for point in polygon["points"]:
            polygon_list.append([point["x"], point["y"]])
        image_list.append({"label": "car", "points": polygon_list, 'description': '','group_id': None,
   'shape_type': 'polygon',
   'flags': {},
   'mask': None})
    labelbox_polygons_formatted.append(image_list)
labelbox_polygons_formatted

In [ ]:
# if not rounding to 13 digits, add trailing zeros

labelme_polygons[0]

In [ ]:
from typing import List, Optional
from pydantic import BaseModel

class Point(BaseModel):
    coords: float

class Car(BaseModel):
    label: str
    points: List[List[float]]
    group_id: Optional[int] = None
    description: Optional[str] = None
    shape_type: str
    flags: dict
    mask: Optional[str] = None

class Image(BaseModel):
    annotations: List[Car]

class ImageCollection(BaseModel):
    images: List[Image]
    

for i in range(len(labelme_polygons)): 
    parsed_data = Image(annotations=labelme_polygons[i])

for j in range(len(labelbox_polygons_formatted)):
    parsed_data = Image(annotations=labelbox_polygons_formatted[j])



In [ ]:
labelme_polygons

In [ ]:
# put both formats in original 
labelme_final= [{"version": "5.4.1",
  "flags": {},
  "shapes": x, "imageData":None, "imagePath":None} for x in labelme_polygons]
labelme_final

In [ ]:
len(labelme_final)

In [ ]:
labelme_final_with_image_data= [{"version": "5.4.1",
  "flags": {},
  "shapes": x, "imageData":labelme_dicts[i]["imageData"], "imagePath":labelme_dicts[i]["imagePath"],"imageHeight":10000, "imageWidth":10000} for i, x in enumerate(labelme_polygons)]

In [ ]:
set([x["imagePath"] for x in labelme_dicts])

In [ ]:
[x["imagePath"][:100] for x in labelme_final_with_image_data]

In [ ]:
labelbox_final= [{"version": "5.4.1",
  "flags": {},
  "shapes": x, "imageData":None, "imagePath":None,"imageHeight":10000, "imageWidth":10000} for x in labelbox_polygons_formatted]
labelbox_final

In [ ]:
# add imagePath to labelbox_final
for i in range(len(labelbox_final)):
    labelbox_final[i]["imagePath"]= os.listdir(labelbox_dir)[i]

labelbox_final[0]

In [ ]:
from PIL import Image
import io
import base64

# Path to your TIFF file

# Open the image using Pillow


# Path to your image file
def encode_image_data(path): 
        
    # Read the image file in binary mode
    with Image.open(f"labelbox_images/{path}") as img:
        print(path)
        # Convert the image to bytes
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        image_data = buffered.getvalue()
    
        # Encode the data to base64
        encoded_string = base64.b64encode(image_data)

# Convert to a regular string (optional, depending on your needs)
        encoded_string = encoded_string.decode('utf-8')
        return encoded_string


for i in range(len(labelbox_final)):
    labelbox_final[i]["imageData"]= encode_image_data(labelbox_dicts[i]["data_row"]["external_id"].replace(".jpeg", ".png"))

labelbox_final_with_image_data= labelbox_final

In [ ]:
[x["imageData"][:100] for x in labelbox_final_with_image_data]

In [ ]:
labelme_final_with_image_data[0]["imageData"][:100]

In [ ]:
labelme_final_with_image_data[0]["imagePath"]

In [ ]:
# join both formats
final = labelme_final_with_image_data + labelbox_final_with_image_data
len(final)

In [ ]:
import re
# write to json
import json
j= 0
for i in final:
    # get only filename after last slash
    print(final[j]["imagePath"])
    filename= re.search(r'[^/]+$', final[j]["imagePath"].replace("\\","/")).group(0)
    print(filename)
    with open(f'labelled_images/{filename}.json', 'w') as f:
        json.dump(i, f)
    j+=1

In [ ]:
# decode 

import base64

# Your base64 encoded string
encoded_string = final[0]["imageData"]

# Decode the base64 string
image_data = base64.b64decode(encoded_string)

#Write the image data to a file
with open('output.png', 'wb') as file:
    file.write(image_data)